In [129]:
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np
from matplotlib import pyplot as plt
from tabulate import tabulate
import os
import pandas as pd
import json
import dataframe_image as dfi
from forwardFv1 import wm_au_emb
from detectFv1 import *
from recoveryFv1 import *
from sklearn.metrics import precision_score,recall_score

In [3]:
root_dir = "D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images"
list_paths = []
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        list_paths.append(os.path.join(subdir, file))

In [4]:
print(list_paths)

['D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\airplane.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\baboon.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\house.tiff', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\lena.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\peppers.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\sailboat.tiff', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\splash.tiff', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\tiffany.bmp']


In [16]:
list_img = []
for i in range(len(list_paths)):
    img = cv2.imread(list_paths[i])
    list_img.append(img)

In [15]:
key = (19,3,137)
list_w_img_19_3_137=[]
for i in range(len(list_paths)):
    img = cv2.imread(list_paths[i])
    w_img = wm_au_emb(img,key=key)
    list_w_img_19_3_137.append(w_img)

In [19]:
for i in range(len(list_img)):
    psnr_13 = cv2.PSNR(list_img[i],list_w_img_13_17_19[i])
    proposed_PSNR_13_17_19.append(psnr_13)
    psnr_3 = cv2.PSNR(list_img[i],list_w_img_3_7_11[i])
    proposed_PSNR_3_7_11.append(psnr_3)
    psnr_19 = cv2.PSNR(list_img[i],list_w_img_19_3_137[i])
    proposed_PSNR_19_3_137.append(psnr_19)

    ssim_13 = ssim(cv2.cvtColor(list_img[i],cv2.COLOR_BGR2GRAY),cv2.cvtColor(list_w_img_13_17_19[i],cv2.COLOR_BGR2GRAY))
    proposed_ssim_13_17_19.append(ssim_13)
    ssim_3 = ssim(cv2.cvtColor(list_img[i],cv2.COLOR_BGR2GRAY),cv2.cvtColor(list_w_img_3_7_11[i],cv2.COLOR_BGR2GRAY))  
    proposed_ssim_3_7_11.append(ssim_3)
    ssim_19 = ssim(cv2.cvtColor(list_img[i],cv2.COLOR_BGR2GRAY),cv2.cvtColor(list_w_img_19_3_137[i],cv2.COLOR_BGR2GRAY))
    proposed_ssim_19_3_137.append(ssim_19)


In [34]:
p_PSNR_round = [round(e,3) for e in proposed_PSNR_19_3_137]
p_PSNR_round.append(round(np.mean(proposed_PSNR_19_3_137),3))

In [32]:
def highlight_max(s):
    # Get 5 largest values of the column
    is_large1 = s.nlargest(1).values
    is_large2 = s.nlargest(2).values
    # Apply style is the current value is among the 5 biggest values
    return ['color: DarkRed;font-weight:900' if v in is_large1 else 'font-weight:900' if v in is_large2 else '' for v in s]

In [38]:
hl_df_ssim = df_all_ssim.style.apply(highlight_max,subset=df_all_ssim.columns[1:],axis = 1).apply(lambda x: ['background: lightgreen' if x.name == 8 else '' for i in x], 
               axis=1)
hl_df_PSNR = df_all_PSNR.style.apply(highlight_max,subset=df_all_PSNR.columns[1:],axis = 1).apply(lambda x: ['background: lightgreen' if x.name == 8 else '' for i in x], 
               axis=1)

In [ ]:
hl_df_ssim.format(precision=4)
hl_df_ssim.hide_index()
hl_df_PSNR.format(precision=4)
hl_df_PSNR.hide_index()

In [43]:
dfi.export(
    hl_df_ssim,
    "results/Comparison_SSIM_Watermarked_Images.png",
    fontsize=12,
)

In [42]:
df_all_PSNR.to_csv("results/PSNR_Watermarked_Images.csv")
df_all_ssim.to_csv("results/SSIM_Watermarked_Images.csv")

In [50]:
N = 512
np.random.seed(1)
t_all_image = np.random.randint(
    low=0, 
    high=256,
    size=(N, N, 3),
    dtype=np.uint8
)

In [128]:
list_tr = [10,20,30,40,50,60,70,80]
list_d = []
for i in range(len(list_tr)):
    list_d.append(int(np.sqrt(512*512*list_tr[i]/100)/2+ 0.5))
print(list_d)

[81, 114, 140, 162, 181, 198, 214, 229]


In [130]:
list_PSNR_Recovered_all = []
list_SSIM_Recovered_all = []
for itr in range(len(list_d)):
    list_PSNR = []
    list_SSIM = []
    for index_img in range(len(list_w_img_19_3_137)):
        t_img = list_w_img_19_3_137[index_img].copy()
        d = list_d[itr]
        t_img[256-d:256+d,256-d:256+d,:] =  t_all_image[256-d:256+d,256-d:256+d,:]
        lv1_matrix = level_one_detection(t_img)
        lv2_matrix = lv1_matrix.copy()
        lv3_matrix = lv1_matrix.copy()
        for channel in range(3):
            lv2_matrix[:,:,channel] = level_two_detection(lv1_matrix[:,:,channel])
            lv3_matrix[:,:,channel] = level_three_detection(lv2_matrix[:,:,channel])
        r1_img,lv4_BGR= stage1_recovery(t_img=t_img,key=(19,3,137))
        r2_img,lv5_BGR = stage2_recovery(r1_img=r1_img,lv4_matrix=lv4_BGR)
        mask1 = np.zeros((512,512),dtype = np.uint8)
        for i in range(512):
            for j in range(512):
                if not lv5_BGR[int(i/1),int(j/1),0]:
                    mask1[i,j] = 255
        mask2 = np.zeros((512,512),dtype = np.uint8)
        for i in range(512):
            for j in range(512):
                if not lv5_BGR[int(i/1),int(j/1),1]:
                    mask2[i,j] = 255
        mask3 = np.zeros((512,512),dtype = np.uint8)
        for i in range(512):
            for j in range(512):
                if not lv5_BGR[int(i/1),int(j/1),2]:
                    mask3[i,j] = 255
        r3_img = t_img.copy()
        r3_img[:,:,0] = cv2.inpaint(r2_img[:,:,0],mask1,3,cv2.INPAINT_TELEA)
        r3_img[:,:,1] = cv2.inpaint(r2_img[:,:,1],mask2,3,cv2.INPAINT_TELEA)
        r3_img[:,:,2] = cv2.inpaint(r2_img[:,:,2],mask3,3,cv2.INPAINT_TELEA)
        list_PSNR.append(cv2.PSNR(r3_img,list_w_img_19_3_137[index_img]))
        list_SSIM.append(ssim(cv2.cvtColor(r3_img,cv2.COLOR_BGR2GRAY),cv2.cvtColor(list_w_img_19_3_137[index_img],cv2.COLOR_BGR2GRAY)))
    list_PSNR_Recovered_all.append(list_PSNR)
    list_SSIM_Recovered_all.append(list_SSIM)

In [83]:
list_image_name =  ['Airplane', 'Baboon', 'House', 'Lena','Peppers','Sailboat','Splash','Tiffany','Average']

In [ ]:
for i in range(8):
    print(list_image_name[i]+","+f"{list_recall_all[i][0]:.5f}"+","+f"{list_recall_all[i][1]:.5f}"+","+f"{list_recall_all[i][2]:.5f}"+","+f"{list_recall_all[i][3]:.5f}")

In [146]:
print(np.mean(list_PSNR_Recovered_all[0]))

39.61632285404647
